In [2]:
%config Completer.use_jedi = False

In [1]:
import aim

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
# Device configuration
device = torch.device('cpu')

# Hyper parameters
num_epochs = 1
num_classes = 10
batch_size = 32
learning_rate = 0.01

session_inst = aim.Session(experiment='ex2')

# aim - Track hyper parameters
session_inst.set_params({
    'num_epochs': num_epochs,
    'num_classes': num_classes,
    'batch_size': batch_size,
    'learning_rate': learning_rate,
}, name='hparams')

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [5]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * 32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 30 == 0:
            print('Epoch [{}/{}], Step [{}/{}], '
                  'Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1,
                                        total_step, loss.item()))

            # aim - Track model loss function
            aim.track(loss.item(), name='loss', epoch=epoch, subset='train')

            correct = 0
            total = 0
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            acc = 100 * correct / total

            # aim - Track metrics
            aim.track(acc, name='accuracy', epoch=epoch, subset='train')

            # TODO: Do actual validation
            if i % 300 == 0:
                aim.track(loss.item(), name='loss', epoch=epoch, subset='val')
                aim.track(acc, name='accuracy', epoch=epoch, subset='val')

Epoch [1/1], Step [1/1875], Loss: 2.3322
Epoch [1/1], Step [31/1875], Loss: 0.3120
Epoch [1/1], Step [61/1875], Loss: 0.8800
Epoch [1/1], Step [91/1875], Loss: 0.3322
Epoch [1/1], Step [121/1875], Loss: 0.2209
Epoch [1/1], Step [151/1875], Loss: 0.1593
Epoch [1/1], Step [181/1875], Loss: 0.1760
Epoch [1/1], Step [211/1875], Loss: 0.1401
Epoch [1/1], Step [241/1875], Loss: 0.0583
Epoch [1/1], Step [271/1875], Loss: 0.2836
Epoch [1/1], Step [301/1875], Loss: 0.7866
Epoch [1/1], Step [331/1875], Loss: 0.1380
Epoch [1/1], Step [361/1875], Loss: 0.1002
Epoch [1/1], Step [391/1875], Loss: 0.1707
Epoch [1/1], Step [421/1875], Loss: 0.0161
Epoch [1/1], Step [451/1875], Loss: 0.0225
Epoch [1/1], Step [481/1875], Loss: 0.0454
Epoch [1/1], Step [511/1875], Loss: 0.3182
Epoch [1/1], Step [541/1875], Loss: 0.1800
Epoch [1/1], Step [571/1875], Loss: 0.0638
Epoch [1/1], Step [601/1875], Loss: 0.1150
Epoch [1/1], Step [631/1875], Loss: 0.0595
Epoch [1/1], Step [661/1875], Loss: 0.0065
Epoch [1/1], Ste

In [ ]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy: {} %'.format(100 * correct / total))

In [7]:
!ls -la

total 32
drwxr-xr-x   6 tjain1  ASV\Domain Users    192 May 18 10:14 .
drwxr-xr-x  18 tjain1  ASV\Domain Users    576 May 17 17:21 ..
drwxr-xr-x   4 tjain1  ASV\Domain Users    128 May 18 10:10 .aim
drwxr-xr-x   3 tjain1  ASV\Domain Users     96 May 18 10:08 .ipynb_checkpoints
-rw-r--r--   1 tjain1  ASV\Domain Users  13793 May 18 10:14 Untitled.ipynb
drwxr-xr-x   3 tjain1  ASV\Domain Users     96 May 18 10:10 data
